In [13]:
import sys,os,os.path
#manual step - hard coded issue
! cd  ../ && {sys.executable} setup.py build && {sys.executable} setup.py install && chmod 777 slim && cd slim && ! sudo {sys.executable} -m pip install -e . && ! echo 'done' && ! ls 



os.environ['PYTHONPATH']='/anaconda/envs/py35/bin/python:/data/home/oneweek/notebooks/models/research:/data/home/oneweek/notebooks/models/research/slim'
print (os.environ['PYTHONPATH'])

import sys
print(sys.executable)

import os
print(os.getcwd().replace('object_detection/fish_detection',''))
sys.path.append(os.getcwd().replace('object_detection/fish_detection',''))
sys.path.append(os.getcwd().replace('/fish_detection',''))
print(sys.path)


/anaconda/envs/py35/bin/python: can't open file 'setup.py': [Errno 2] No such file or directory
/anaconda/envs/py35/bin/python:/data/home/oneweek/notebooks/models/research:/data/home/oneweek/notebooks/models/research/slim
/anaconda/envs/py35/bin/python
/data/home/azfishvm/notebooks/FindingFishwithTensorflowObjectdetectionAPI/models/research/
['', '/opt/caffe/python', '/opt/caffe2/build', '/data/home/azfishvm/notebooks/FindingFishwithTensorflowObjectdetectionAPI/models/research/object_detection/fish_detection', '/anaconda/envs/py35/lib/python35.zip', '/anaconda/envs/py35/lib/python3.5', '/anaconda/envs/py35/lib/python3.5/plat-linux', '/anaconda/envs/py35/lib/python3.5/lib-dynload', '/home/azfishvm/.local/lib/python3.5/site-packages', '/anaconda/envs/py35/lib/python3.5/site-packages', '/anaconda/envs/py35/lib/python3.5/site-packages/dlib-19.15.0-py3.5-linux-x86_64.egg', '/anaconda/envs/py35/lib/python3.5/site-packages/xgboost-0.80-py3.5.egg', '/anaconda/envs/py35/lib/python3.5/site-packa

In [14]:
!  echo $PYTHONPATH

/anaconda/envs/py35/bin/python:/data/home/oneweek/notebooks/models/research:/data/home/oneweek/notebooks/models/research/slim


In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import io
import logging
import os

from lxml import etree
import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

In [16]:
def dict_to_tf_example(data,
                       dataset_directory,
                       label_map_dict,
                       ignore_difficult_instances=False,
                       image_subdirectory='JPEGImages'):
  """Convert XML derived dict to tf.Example proto.

  Notice that this function normalizes the bounding box coordinates provided
  by the raw data.

  Args:
    data: dict holding PASCAL XML fields for a single image (obtained by
      running dataset_util.recursive_parse_xml_to_dict)
    dataset_directory: Path to root directory holding PASCAL dataset
    label_map_dict: A map from string label names to integers ids.
    ignore_difficult_instances: Whether to skip difficult instances in the
      dataset  (default: False).
    image_subdirectory: String specifying subdirectory within the
      PASCAL dataset directory holding the actual image data.

  Returns:
    example: The converted tf.Example.

  Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
  """

   
  #img_path = os.path.join(dataset_directory, image_subdirectory, data['filename'])
  img_path = data['path']
  print('img_path',img_path)
  full_path = img_path
  print('full_path',full_path)  
  with tf.gfile.GFile(full_path, 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)
  if image.format != 'JPEG':
    raise ValueError('Image format not JPEG')
  key = hashlib.sha256(encoded_jpg).hexdigest()

  width = int(data['size']['width'])
  height = int(data['size']['height'])

  xmin = []
  ymin = []
  xmax = []
  ymax = []
  classes = []
  classes_text = []
  truncated = []
  poses = []
  difficult_obj = []
  if 'object' in data:
    for obj in data['object']:
      #print('obj',obj)
      #difficult = bool(int(obj['difficult']))
      #if ignore_difficult_instances and difficult:
        #continue
      difficult  =False
      difficult_obj.append(int(difficult))

      xmin.append(float(obj['bndbox']['xmin']) / width)
      ymin.append(float(obj['bndbox']['ymin']) / height)
      xmax.append(float(obj['bndbox']['xmax']) / width)
      ymax.append(float(obj['bndbox']['ymax']) / height)
      classes_text.append(obj['name'].encode('utf8'))
      classes.append(label_map_dict[obj['name']])
      truncatedflag=False
      truncated.append(int(truncatedflag))
      poses.append(obj['pose'].encode('utf8'))

  example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(
          data['filename'].encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(
          data['filename'].encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
      'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
      'image/object/truncated': dataset_util.int64_list_feature(truncated),
      'image/object/view': dataset_util.bytes_list_feature(poses),
  }))
  return example

In [17]:
def main():
  
   
    ignore_difficult_instances=False
    
    writer = tf.python_io.TFRecordWriter(output_path)

    label_map_dict = label_map_util.get_label_map_dict(label_map_path)

    examples_path = os.path.join(data_dir, 'ImageSets', 'Main',
                                 'fish_'+flag_set+'.txt')
    
    annotations_dir = os.path.join(data_dir,'Annotations' )
    examples_list = dataset_util.read_examples_list(examples_path)
    
    for idx, example in enumerate(examples_list):
        
        if example=='':
            print('invalid filename','idx:',idx,'example:',example)
            continue
            
        if idx % 100 == 0:
            logging.info('On image %d of %d', idx, len(examples_list))
        
        path = os.path.join(annotations_dir, example + '.xml')
        print(path)
        with tf.gfile.GFile(path, 'r') as fid:
            xml_str = fid.read()
        xml = etree.fromstring(xml_str)
        data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
       
        print('data_dir',data_dir)
        print('label_map_dict',label_map_dict)
        print('path',data['path'])
        relativepath=data['path'].replace('C:\\lab\\tensorflowobjectdetection\\fishdata\\Videos\\video1_output\\', '').replace('\\','/')
        print('relativepath',relativepath)
        print('path',data_dir+'/'+relativepath)
        data['path']=data_dir+'/'+relativepath
        print('data',data)
        tf_example = dict_to_tf_example(data, data_dir, label_map_dict,ignore_difficult_instances)

        writer.write(tf_example.SerializeToString())
        
    writer.close()




In [18]:
! ls fishdata/Videos/video1_output


Annotations  JPEGImages  pascal_label_map.pbtxt  positive
ImageSets    negative	 pascal_val.record	 testImages


In [19]:
data_dir = "fishdata/Videos/video1_output"

label_map_path="fishdata/Videos/video1_output/pascal_label_map.pbtxt"


In [20]:
output_path="fishdata/Videos/video1_output/pascal_val.record"
flag_set='val'
main()

fishdata/Videos/video1_output/Annotations/video1_frame_9.xml
data_dir fishdata/Videos/video1_output
label_map_dict {'fish': 1}
path C:\lab\tensorflowobjectdetection\fishdata\Videos\video1_output\JPEGImages\video1_frame_9.jpg
relativepath JPEGImages/video1_frame_9.jpg
path fishdata/Videos/video1_output/JPEGImages/video1_frame_9.jpg
data {'filename': 'video1_frame_9', 'path': 'fishdata/Videos/video1_output/JPEGImages/video1_frame_9.jpg', 'folder': 'Annotation', 'segmented': '0', 'size': {'depth': '3', 'height': '336', 'width': '600'}, 'source': {'database': 'Unknown'}}
img_path fishdata/Videos/video1_output/JPEGImages/video1_frame_9.jpg
full_path fishdata/Videos/video1_output/JPEGImages/video1_frame_9.jpg
fishdata/Videos/video1_output/Annotations/video1_frame_13.xml
data_dir fishdata/Videos/video1_output
label_map_dict {'fish': 1}
path C:\lab\tensorflowobjectdetection\fishdata\Videos\video1_output\JPEGImages\video1_frame_13.jpg
relativepath JPEGImages/video1_frame_13.jpg
path fishdata/Vi

/data/home/azfishvm/notebooks/FindingFishwithTensorflowObjectdetectionAPI/models/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


In [21]:
output_path="fishdata/Videos/video1_output/pascal_train.record"
flag_set='train'
main()

invalid filename idx: 0 example: 
invalid filename idx: 1 example: 
invalid filename idx: 2 example: 
invalid filename idx: 3 example: 
invalid filename idx: 4 example: 
invalid filename idx: 5 example: 
invalid filename idx: 6 example: 
fishdata/Videos/video1_output/Annotations/video1_frame_1.xml
data_dir fishdata/Videos/video1_output
label_map_dict {'fish': 1}
path C:\lab\tensorflowobjectdetection\fishdata\Videos\video1_output\JPEGImages\video1_frame_1.jpg
relativepath JPEGImages/video1_frame_1.jpg
path fishdata/Videos/video1_output/JPEGImages/video1_frame_1.jpg
data {'filename': 'video1_frame_1', 'path': 'fishdata/Videos/video1_output/JPEGImages/video1_frame_1.jpg', 'folder': 'Annotation', 'segmented': '0', 'size': {'depth': '3', 'height': '336', 'width': '600'}, 'source': {'database': 'Unknown'}}
img_path fishdata/Videos/video1_output/JPEGImages/video1_frame_1.jpg
full_path fishdata/Videos/video1_output/JPEGImages/video1_frame_1.jpg
fishdata/Videos/video1_output/Annotations/video1_

/data/home/azfishvm/notebooks/FindingFishwithTensorflowObjectdetectionAPI/models/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


fishdata/Videos/video1_output/Annotations/video1_frame_62.xml
data_dir fishdata/Videos/video1_output
label_map_dict {'fish': 1}
path C:\lab\tensorflowobjectdetection\fishdata\Videos\video1_output\JPEGImages\video1_frame_62.jpg
relativepath JPEGImages/video1_frame_62.jpg
path fishdata/Videos/video1_output/JPEGImages/video1_frame_62.jpg
data {'filename': 'video1_frame_62', 'path': 'fishdata/Videos/video1_output/JPEGImages/video1_frame_62.jpg', 'folder': 'Annotation', 'segmented': '0', 'size': {'depth': '3', 'height': '336', 'width': '600'}, 'source': {'database': 'Unknown'}}
img_path fishdata/Videos/video1_output/JPEGImages/video1_frame_62.jpg
full_path fishdata/Videos/video1_output/JPEGImages/video1_frame_62.jpg
fishdata/Videos/video1_output/Annotations/video1_frame_63.xml
data_dir fishdata/Videos/video1_output
label_map_dict {'fish': 1}
path C:\lab\tensorflowobjectdetection\fishdata\Videos\video1_output\JPEGImages\video1_frame_63.jpg
relativepath JPEGImages/video1_frame_63.jpg
path fis